In [ ]:
# ===============================================================
#             YouTube Content Automation Script
# ===============================================================

# STEP A: Install all the required libraries quietly
!pip install -q google-generativeai gTTS requests Pillow moviepy
print("Success: All required libraries are installed.")

# STEP B: Import libraries and configure API Keys
import google.generativeai as genai
import requests
from gtts import gTTS
from moviepy.editor import ImageClip, concatenate_videoclips, AudioFileClip, CompositeVideoClip
from PIL import Image
import os
import json
import time

# --- PASTE YOUR API KEYS HERE ---
GOOGLE_API_KEY = ""
PEXELS_API_KEY = ""


# Configure the Gemini API
try:
    genai.configure(api_key=GOOGLE_API_KEY)
    model = genai.GenerativeModel('gemini-1.5-flash')
except Exception as e:
    print(f"Error configuring Google API: {e}")
    print("Please ensure your GOOGLE_API_KEY is correct.")

# Create directories to store our assets and final output
os.makedirs('assets', exist_ok=True)
os.makedirs('output', exist_ok=True)

# ===============================================================
#          Main Functions for Video Creation
# ===============================================================

def generate_structured_script(topic):
    """Generates a scene-by-scene script with keywords for perfect sync."""
    print("1/5: Generating structured script with Gemini AI...")
    prompt = f"""
    Create content for a short educational YouTube video on the topic: '{topic}'.
    Your response MUST be a valid JSON array of objects.
    Each object in the array represents a scene and must have two keys:
    1. "scene_text": A short paragraph for the voiceover (20-40 words).
    2. "keyword": A simple, one or two-word visual keyword for that specific scene.

    Generate 4 to 6 scenes in total.

    Example Response Format:
    [
      {{"scene_text": "Imagine a particle that can be in two places at once. This is the mind-bending world of quantum physics.", "keyword": "atom particle"}},
      {{"scene_text": "Now, imagine two of these particles linked together in a special way. What happens to one instantly affects the other, no matter the distance.", "keyword": "linked particles"}}
    ]
    """
    try:
        response = model.generate_content(prompt)
        cleaned_response = response.text.strip().replace('```json', '').replace('```', '')
        scenes = json.loads(cleaned_response)
        print("Success: Script generated successfully!")
        return scenes
    except Exception as e:
        print(f"Error: Script generation failed. Details: {e}")
        return None

def generate_scene_assets(scenes):
    """Generates audio and image assets for each scene."""
    print("\n2/5: Generating assets for each scene...")
    asset_paths = []
    headers = {"Authorization": PEXELS_API_KEY}

    for i, scene in enumerate(scenes):
        print(f"  -> Processing Scene {i+1} of {len(scenes)}...")
        try:
            tts = gTTS(text=scene['scene_text'], lang='en', slow=False)
            audio_path = f"assets/audio_{i}.mp3"
            tts.save(audio_path)
        except Exception as e:
            print(f"  Error: Audio generation failed for scene {i+1}. Details: {e}")
            continue

        try:
            url = f"https://api.pexels.com/v1/search?query={scene['keyword']}&per_page=1"
            r = requests.get(url, headers=headers)
            r.raise_for_status()
            data = r.json()
            if not data['photos']:
                print(f"  Warning: No image found for keyword: '{scene['keyword']}'. Skipping scene.")
                continue

            img_url = data['photos'][0]['src']['large2x']
            img_data = requests.get(img_url).content
            image_path = f'assets/image_{i}.jpg'
            with open(image_path, 'wb') as handler:
                handler.write(img_data)

            img = Image.open(image_path)
            img = img.resize((1920, 1080))
            img.save(image_path)
            asset_paths.append({'audio': audio_path, 'image': image_path})
            time.sleep(1)
        except Exception as e:
            print(f"  Error: Image download failed for scene {i+1}. Details: {e}")
            continue

    print("Success: All assets generated!")
    return asset_paths

def assemble_video(asset_paths, output_filename="output/final_video.mp4"):
    """Assembles the final video from scene assets."""
    print("\n3/5: Assembling video clips...")
    clips = []
    for paths in asset_paths:
        try:
            audio_clip = AudioFileClip(paths['audio'])
            image_clip = ImageClip(paths['image']).set_duration(audio_clip.duration)

            image_clip = image_clip.resize(height=1080)
            image_clip = image_clip.resize(lambda t: 1 + 0.02 * t)
            image_clip = image_clip.set_position(('center', 'center'))

            final_clip = CompositeVideoClip([image_clip], size=(1920, 1080)).set_audio(audio_clip)
            clips.append(final_clip)
        except Exception as e:
            print(f"  Error: Failed to create clip. Details: {e}")
            continue

    if not clips:
        print("Error: No clips were created. Cannot assemble video.")
        return

    print("\n4/5: Concatenating clips into final video...")
    final_video = concatenate_videoclips(clips, method="compose")
    final_video.write_videofile(
        output_filename,
        codec='libx264',
        audio_codec='aac',
        fps=24
    )
    print(f"\n5/5: Video generation complete!")
    print(f"Your video is saved at: {output_filename}")


# ===============================================================
#          Execute the Video Creation Pipeline
# ===============================================================
if "YOUR_GOOGLE_AI_API_KEY" in GOOGLE_API_KEY or "YOUR_PEXELS_API_KEY" in PEXELS_API_KEY:
    print("Warning: PLEASE PASTE YOUR API KEYS in the script before running.")
else:
    video_topic = input("Please enter the video topic you want to create: ")

    if video_topic:
        scenes_data = generate_structured_script(video_topic)

        if scenes_data:
            assets = generate_scene_assets(scenes_data)

            if assets:
                assemble_video(assets)
    else:
        print("No topic entered. Exiting script.")